In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
from numpy.random import seed
seed(42)
import pickle

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob 
%matplotlib inline

Some learning parameters

In [20]:
IMG_CHANNELS = 3
#TRAIN_SIZE=80000
TRAIN_SIZE = 80000
BATCH_SIZE = 64
EPOCHS = 30


Data exploration

In [21]:
#I used cropped images
train_labels = pd.read_csv('train_labels.csv')
train_labels['label'].value_counts()


0    130908
1     89117
Name: label, dtype: int64

In [22]:
(train_labels.label.value_counts() / len(train_labels)).to_frame()

,label
0,0.594969
1,0.405031


Balancing the data

In [23]:
train_neg = train_labels[train_labels['label']==0].sample(TRAIN_SIZE,random_state=45)
train_pos = train_labels[train_labels['label']==1].sample(TRAIN_SIZE,random_state=45)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train_data = shuffle(train_data)


In [24]:
train_data['label'].value_counts()

0    80000
1    80000
Name: label, dtype: int64

In [25]:
def append_ext(fn):
    return fn+".tif"

In [26]:
y = train_data['label']
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=45, stratify=y)

print(train_df.shape)
print(valid_df.shape)

(128000, 2)
(32000, 2)


In [27]:
train_df['id'] = train_df['id'].apply(append_ext)
valid_df['id'] = valid_df['id'].apply(append_ext)
train_df.head()

,id,label
53542,87f5894925a0b2d7c157ef7591e3818b81f4fa50.tif,0
46578,e354401762727ee30f82fefb24146d9a20e81899.tif,0
54154,f63aec1b48e26f41ce8a4ad8fab0fda3517b7a90.tif,0
137678,713f6d6a38be9ee7831b142403d85f2261f15056.tif,1
83385,0a712f489695bb37dea4131a7b749b67579b31f5.tif,1


Image generator that load images in batches

In [28]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [29]:
BATCH_SIZE = 64
train_path = './train'
train_df['label'] = train_df['label'].astype(str)
valid_df['label'] = valid_df['label'].astype(str)

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

valid_loader = valid_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)


Found 128000 validated image filenames belonging to 2 classes.
Found 32000 validated image filenames belonging to 2 classes.


In [30]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

2000
500


In [39]:
cnn = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.4),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.3),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.2),
    BatchNormalization(),

    Flatten(),
     Dense(128, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='sigmoid')
])


Round one training

In [40]:
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [42]:
h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.00001)


h3 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2000/2000 [==============================] - ETA: 0s - loss: 0.4929 - accuracy: 0.7700 - auc_2: 0.8415WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x17b74df70> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export 

2000/2000 [==============================] - 130s 65ms/step - loss: 0.2468 - accuracy: 0.8972 - auc_2: 0.9627 - val_loss: 0.3250 - val_accuracy: 0.8651 - val_auc_2: 0.9394
Epoch 6/30
2000/2000 [==============================] - 131s 65ms/step - loss: 0.2458 - accuracy: 0.8968 - auc_2: 0.9631 - val_loss: 0.3387 - val_accuracy: 0.8586 - val_auc_2: 0.9351
Epoch 7/30
2000/2000 [==============================] - 131s 66ms/step - loss: 0.2441 - accuracy: 0.8978 - auc_2: 0.9636 - val_loss: 0.3213 - val_accuracy: 0.8683 - val_auc_2: 0.9407
Epoch 8/30
2000/2000 [==============================] - 1148s 574ms/step - loss: 0.2437 - accuracy: 0.8978 - auc_2: 0.9637 - val_loss: 0.3449 - val_accuracy: 0.8584 - val_auc_2: 0.9347
Epoch 9/30
2000/2000 [==============================] - 1062s 531ms/step - loss: 0.2412 - accuracy: 0.8993 - auc_2: 0.9644 - val_loss: 0.3241 - val_accuracy: 0.8657 - val_auc_2: 0.9399
Epoch 10/30
2000/2000 [==============================] - 131s 66ms/step - loss: 0.2401 - acc

2000/2000 [==============================] - 130s 65ms/step - loss: 0.2151 - accuracy: 0.9115 - auc_2: 0.9717 - val_loss: 0.3347 - val_accuracy: 0.8671 - val_auc_2: 0.9397
Epoch 20/30
2000/2000 [==============================] - 129s 65ms/step - loss: 0.2135 - accuracy: 0.9118 - auc_2: 0.9721 - val_loss: 0.3340 - val_accuracy: 0.8679 - val_auc_2: 0.9398
Epoch 21/30
2000/2000 [==============================] - 130s 65ms/step - loss: 0.2147 - accuracy: 0.9113 - auc_2: 0.9718 - val_loss: 0.3351 - val_accuracy: 0.8671 - val_auc_2: 0.9395
Epoch 22/30
2000/2000 [==============================] - 130s 65ms/step - loss: 0.2144 - accuracy: 0.9111 - auc_2: 0.9719 - val_loss: 0.3356 - val_accuracy: 0.8670 - val_auc_2: 0.9394
Epoch 23/30
2000/2000 [==============================] - 1031s 516ms/step - loss: 0.2137 - accuracy: 0.9113 - auc_2: 0.9721 - val_loss: 0.3353 - val_accuracy: 0.8674 - val_auc_2: 0.9398
Epoch 24/30
2000/2000 [==============================] - 1029s 515ms/step - loss: 0.2146 -

Final outcome 95.88%, historical highest 96.02%